# Part1

## (a) 
Compute the eigenvalues and eigenvectors of A.

In [1]:
import numpy as np

# Define the system matrix for h1(t) and h2(t)
A = np.array([[-1, 0],
              [0, -10]])

# Compute eigenvalues and eigenvectors of the matrix A
eigenvalues, eigenvectors = np.linalg.eig(A)

eigenvalues, eigenvectors

(array([ -1., -10.]),
 array([[1., 0.],
        [0., 1.]]))

## (b)
Compute the leading spatial POD mode.

In [2]:
# Constants and parameters for simulation
dt = 1e-3  # Time step
total_time = 1  # Total time of simulation
num_steps = int(total_time / dt)  # Number of time steps
num_trajectories = 1000  # Number of trajectories

# Initial conditions
h_initial = np.zeros((2, 1))

# Discrete-time system matrix
A_discrete = np.eye(2) + A * dt
B_discrete = np.array([[1e-3], [1e3]]) * dt

# To store all trajectories
all_trajectories = np.zeros((2, num_steps, num_trajectories))

# Simulate the system
np.random.seed(42)  # for reproducibility
for j in range(num_trajectories):
    h = h_initial.copy()
    for i in range(num_steps):
        x_t = np.random.normal(0, 1)  # Sample x(t)
        h = A_discrete @ h + B_discrete * x_t  # Euler integration step
        all_trajectories[:, i, j] = h.squeeze()

# Perform POD via time-averaging across all trajectories
data_matrix = all_trajectories.reshape(2, -1)  # Flatten trajectories into a matrix of 2 x (num_steps*num_trajectories)
covariance_matrix = np.cov(data_matrix)  # Compute the covariance matrix
pod_eigenvalues, pod_eigenvectors = np.linalg.eig(covariance_matrix)  # Eigen-decomposition

# Sort eigenvectors based on eigenvalues in descending order
sorted_indices = np.argsort(-pod_eigenvalues)
leading_pod_mode = pod_eigenvectors[:, sorted_indices[0]]

pod_eigenvalues, pod_eigenvectors, leading_pod_mode


(array([1.47942103e-10, 4.80789523e+01]),
 array([[-1.00000000e+00, -1.74345659e-06],
        [ 1.74345659e-06, -1.00000000e+00]]),
 array([-1.74345659e-06, -1.00000000e+00]))

## (h)
Compute the $W_c(t)$ and its eigenvalues.

In [3]:
import numpy as np
from scipy.linalg import expm, eigvals

# Define the system matrices
A = np.array([[-1, 0],
              [0, -10]])
B = np.array([[1e-3],
              [1e3]])

# Time at which to compute the Gramian
t = 1

# Compute the controllability Gramian
# Wc(t) = integral from 0 to t of expm(A*s) * B * B.T * expm(A.T*s) ds
# For numerical computation, we use the matrix exponential at discrete steps

# Discretization parameters
num_steps = 1000
delta_t = t / num_steps
Wc = np.zeros((2, 2))

for step in range(num_steps):
    tau = step * delta_t
    Wc += expm(A * tau) @ B @ B.T @ expm(A.T * tau) * delta_t

# Compute the eigenvalues of the controllability Gramian
eigenvalues = eigvals(Wc)

Wc, eigenvalues

(array([[4.32764835e-07, 9.14084809e-02],
        [9.14084809e-02, 5.05016666e+04]]),
 array([2.67318683e-07+0.j, 5.05016666e+04+0.j]))

## (i)
Compute the $\tilde{A}$ and $\tilde{B}$, then compute the $\tilde{W}_c(1)$.

In [4]:
# Define the transformation matrix T
T = np.array([[1e3, 0],
              [0, 1e-3]])

# Compute the new A and B matrices for the transformed system
A_tilde = T @ A @ np.linalg.inv(T)
B_tilde = T @ B

# Compute the controllability Gramian for the transformed system
Wc_tilde = np.zeros((2, 2))

for step in range(num_steps):
    tau = step * delta_t
    Wc_tilde += expm(A_tilde * tau) @ B_tilde @ B_tilde.T @ expm(A_tilde.T * tau) * delta_t

# Compute the eigenvalues of the controllability Gramian
eigenvalues_tilde = eigvals(Wc_tilde)

Wc_tilde, eigenvalues_tilde

(array([[0.43276483, 0.09140848],
        [0.09140848, 0.05050167]]),
 array([0.45349829+0.j, 0.02976822+0.j]))